We'll at least start out with the given guys before feature engineering.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import RFECV #was hoping for this without the reverse, but I don't see it. Oh well when
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
#it's just 8; well, it'll be more hectic momentarily...

# from sklearn.metrics import make_scorer
from sklearn.metrics import root_mean_squared_log_error #Phew this on is on there... I mean find if we'd have to square,
#but rather not have to write the formula.

# from sklearn.metrics import make_scorer

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

In [6]:
from sklearn.preprocessing import PolynomialFeatures

In [62]:
#The following is in case I need it later. As, it seems quite barbaric to sort through the list each one... however this would require
#changing the numbers into a dataframe.

def log_prepper(num):
    return min(0, num)

In [3]:
df = pd.read_csv('../data/cleaned_train.csv')
print(df.shape)
df.head()

(90609, 9)


,sex,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [4]:
#I honestly forgot to earlier, although I do like the data is preserved: At least at the moment we also need to dummify sex
#to get our lovely sklearn methods to work:

df = pd.get_dummies(df, columns=['sex'], dtype=int, drop_first=True)
df.head()  #Note that the baseline, ie 0s in the rest, will be indicativ of female abalones.

,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings,sex_I,sex_M
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,0,0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,0,0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,1,0
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,0,1
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,1,0


In [5]:
X = df.drop('rings', axis=1)
y = df['rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26) #Relatively large test size as we
#have so much data to work with...
#Re. 26 - I like that number. Discussion not for now

mms = MinMaxScaler() #Temp. removed to have an easier time seeing which feature names were used.
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

Given that this is somewhat of a professional environment, as opposed to my boot camp I just finished, I won't bother to explain all of my thoughts. However, note that I'll reuse this and similar lines of code until finding desired results.

In [6]:
# model_defaults.get_feature_names_out()

#Hmm, quite interesting that we chose all the weights.... ie I'd understand if one, but
#apparetly the relative discremencies upon focusing on less and less of the alabone, quite literally, is that useful.

#Also, the distinguishment between female is somehwat negligible, despite hte many difference we obsered... however from
#female to on-binary remains significant.

In [7]:
#Oh, just remembered we're supposed to use MSLE. At th least we have a benchmark of what's good 'objectively'

lr = LinearRegression()
rfe = RFECV(lr, step=1, cv=5,
                       min_features_to_select=3)

model_defaults = rfe.fit(X_train, y_train)

preds_model_defaults = rfe.predict(X_test)#And we're somehow getting some negative predictions.... note that againt his is
#without any feature engineering.... Regardless, we'll just brute force it. Unsure if we have a better wayt han the following:

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
print(model_defaults.cv_results_['mean_test_score'])
root_mean_squared_log_error(y_test, preds_model_defaults)

[]
0.6062234959016377 0.6034787674182922
[0.59003897 0.59367525 0.59568496 0.59954274 0.59970396 0.60580828
 0.60580744]


0.16508650876858538

In [8]:
#A little tempted to play around with the defaults, but eh...consideing that we likely need to engineer some things...

rf = RandomForestRegressor(random_state=26)

##Seems a bit redundant to use both here...
# rfe = RFECV(rf, step=1, cv=5, 
#                        min_features_to_select=3)

model_defaults = rf.fit(X_train, y_train)

preds_model_defaults = rf.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
# print(cross_val_score(rf, 
#                       X_train, y_train, cv = 5).mean()) #Yeah,c ommenting this guy out... taking too long plus a bit redundant
# #Given that random forests are already aggregativing everything
root_mean_squared_log_error(y_test, preds_model_defaults)

#oooooh yesss this is quite juicy... quite juicy abalane 

0.9504620112507841 0.6557858728716266


0.15345037771109488

In [23]:
bag = BaggingRegressor(random_state=26)

##Seems a bit redundant to use both here...
# rfe = RFECV(rf, step=1, cv=5, 
#                        min_features_to_select=3)

model_defaults = bag.fit(X_train, y_train)

preds_model_defaults = bag.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
# print(cross_val_score(rf, 
#                       X_train, y_train, cv = 5).mean()) #Yeah,c ommenting this guy out... taking too long plus a bit redundant
# #Given that random forests are already aggregativing everything
root_mean_squared_log_error(y_test, preds_model_defaults)

0.9313299951392628 0.6238598931844348


0.16064598506264355

In [20]:
# pipe = Pipeline([
#     ('svr', SVR())
# ])
# pipe_params = {'svr__kernel' : ['rbf'] #Eh, taking too long to do all of them... just go with rbf for now
#                ,'svr__C' : [.9, .95 , 1]
#               }
# gs = GridSearchCV(pipe,
#                   param_grid=pipe_params,
#                   cv=5)

# gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('svr', SVR())]),
             param_grid={'svr__C': [0.9, 0.95, 1], 'svr__kernel': ['rbf']})

In [21]:
# preds_model_defaults = rf.predict(X_test)
# for i in range(len(preds_model_defaults)):
#     if preds_model_defaults[i] < 0:
#         preds_model_defaults[i] = 0

# print(gs.score(X_train, y_train), gs.score(X_test, y_test))
# print(gs.best_score_)
# print(gs.best_params_)
        
# root_mean_squared_log_error(y_test, preds_model_defaults)

##0.6187870521022931 0.6175574607103471 #Commenting out as it took way too long... note that it has the same RMSLE as forests
## 0.6167895215540445
## {'svr__C': 1, 'svr__kernel': 'rbf'}
## 0.15345037771109488

0.6187870521022931 0.6175574607103471
0.6167895215540445
{'svr__C': 1, 'svr__kernel': 'rbf'}


0.15345037771109488

So, bottom line we 'like' the accuracy and processing time of Random Forests. Perhaps regression is just that intense, or I'm underappeicating how giant this dataset is, but SVR took forever. So, likely we'll jump just do Forests as we move on. We'll see.

## The Start of Feature Engineering: Polynomials

We'll keep things PG and just go up to squared terms, naturally considering interaction effects as well.

In [64]:
X = df.drop('rings', axis=1)
y = df['rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26)

poly = PolynomialFeatures(degree = 2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

df_poly_train = pd.DataFrame(X_train_poly, columns=poly.get_feature_names_out(X.columns)) #We'll use this later
df_poly_test = pd.DataFrame(X_test_poly, columns=poly.get_feature_names_out(X.columns))

mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [11]:
len(poly.get_feature_names_out(X.columns))

54

In [36]:
#Hmm, so a maximum of 54 features now. let's do this!

lr = LinearRegression()
rfe = RFECV(lr, step=1, cv=5,
                       min_features_to_select=35)

model_defaults = rfe.fit(X_train_poly, y_train)

preds_model_defaults = rfe.predict(X_test_poly)#And we're somehow getting some negative predictions.... note that againt his is
#without any feature engineering.... Regardless, we'll just brute force it. Unsure if we have a better wayt han the following:

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train_poly, y_train), model_defaults.score(X_test_poly, y_test))
print(model_defaults.cv_results_['mean_test_score'])
root_mean_squared_log_error(y_test, preds_model_defaults)

[]
0.637502643156102 0.6391210169168662
[0.61664033 0.61906504 0.62020718 0.62345992 0.62343427 0.62558104
 0.62686518 0.6280259  0.62854671 0.62875105 0.62839985 0.62840971
 0.6287009  0.62930472 0.62911993 0.62926181 0.62897665 0.62941128
 0.62974946 0.62969177 0.6300942  0.63044306 0.63204007 0.63281593
 0.6332747  0.63349061 0.6335524  0.63323694 0.633351   0.633187
 0.63318795 0.63320335 0.63326952 0.63328497 0.63329516 0.63333263
 0.63334052 0.63339293 0.63334014 0.6333579  0.63336914 0.63336319
 0.63336625 0.63336738 0.63336738]


0.1562207342973044

In [44]:
print(model_defaults.n_features_)
model_defaults.get_feature_names_out()

36


array(['x0', 'x1', 'x2', 'x3', 'x4', 'x6', 'x9', 'x10', 'x11', 'x12',
       'x13', 'x14', 'x15', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23',
       'x26', 'x28', 'x29', 'x30', 'x31', 'x34', 'x35', 'x36', 'x39',
       'x40', 'x41', 'x42', 'x44', 'x45', 'x46', 'x48', 'x49'],
      dtype=object)

In [58]:
#Likely some better method exists, but eh...

targets = []
for i in model_defaults.get_feature_names_out():
    targets.append(int(i[1:]))
target_columns = []
for i in targets:
    target_columns.append(df_poly_train.columns[i])

In [60]:
target_columns[:6] #Hmm, so on a pure scale we are disregarding the second weight as well as pure sex variables. Howeve,r htey
#do manifest in conjecture with others.

#Recall earlier how we proposed that inherently, likely, the sex is meaningless and just a proxy for expected weights. So,
#perhaps this is inline with what we assumed.

['length',
 'diameter',
 'height',
 'whole_weight',
 'whole_weight1',
 'shell_weight']

Recall that the pure linear rmsle with lr was 0.16508650876858538.

Hmm, originally I'm doing this after min/max scaling...yeah that's probably not a move as perhaps the scaling is interfering with their effects. Oh, while all that is true I had a more fundamental mistake - NOOB remember to score around the POLY dataset and not the original one...

There we go, at 50 we already have a much lower RMSLE of .156. Let's keep on going for a while.

In [ ]:
#I'll leave the name the same, so note how now it's just the subset per the earlier linear regression.

X_train_poly = np.array(df_poly_train[target_columns])
X_test_poly = np.array(df_poly_test[target_columns])

In [77]:
rf = RandomForestRegressor(random_state=26)

model_defaults = rf.fit(X_train_poly, y_train)

preds_model_defaults = rf.predict(X_test_poly)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train_poly, y_train), model_defaults.score(X_test_poly, y_test))
# print(cross_val_score(rf, 
#                       X_train, y_train, cv = 5).mean()) #Yeah,c ommenting this guy out... taking too long plus a bit redundant
# #Given that random forests are already aggregativing everything
root_mean_squared_log_error(y_test, preds_model_defaults)

#A fair bit lower... Hmm. My intuition tells me to even keep on going for now and get to logs first. So, perhaps
#we'll return and actually tune the forest's parameters.

#Tempted to do SVR just cause I need to be afk for a while...yeah why not. Let's do it.

0.9500471760962943 0.6566633589144847


0.15307428574200113

In [ ]:
#Potential future spot for bagged forests x polys.

In [75]:
X_train_poly = np.array(df_poly_train[target_columns])
X_test_poly = np.array(df_poly_test[target_columns])

(22653, 36)

In [76]:
svr = SVR() #Going with defautls at least for now

svr = svr.fit(X_train_poly, y_train)

preds_model_defaults = svr.predict(X_test_poly)
for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(svr.score(X_train_poly, y_train), svr.score(X_test_poly, y_test))
        
root_mean_squared_log_error(y_test, preds_model_defaults)

#Aww, all that waiting and not even better than the forests.

0.6256051790865003 0.6257469825562314


0.1549904012357784

So, we'e made a bit of progress with introducing polynomial degree 2 and interaction effects. While RF is still our best and preferred model, we're down from .15345 to .15307 RMSLE.

## Round II - Logs

As we predicted we'd have to do, given assumptions in using a log based metric (first time using such a thing so I honestly don't know if it's grounded anywhere. We'd do it anyways, but still...)

In [9]:
#Some notes in case I need them

In [11]:
# pipe = Pipeline([
#     ('cvec', CountVectorizer(lowercase=False)),
#     ('bc', BaggingClassifier(random_state=26))
# ])
# pipe_params = {'cvec__min_df' : [0.0]
#                ,'cvec__max_df' : [.95]
#                ,'cvec__ngram_range' : [(1,2)]
#                ,'cvec__max_features' : [1800, 1750, 1700]
#                ,'cvec__stop_words' : ['english']
#               }
# gs = GridSearchCV(pipe,
#                   param_grid=pipe_params,
#                   cv=5)

# gs.fit(X_train, y_train)

In [12]:
# print(gs.score(X_train, y_train), gs.score(X_test, y_test))
# print(gs.best_score_)
# print(gs.best_params_)
# #Oh wow, this did a lot 